# **Fruit and Vegetable Image Classification Project**

This project involves building and training a **Convolutional Neural Network (CNN)** model to classify images of fruits and vegetables. The dataset is divided into training, validation, and test sets. The goal is to preprocess the images, build the CNN model, train it, evaluate its performance, and visualize the results.

Dataset: https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition

## **Importing Necessary Libraries**

We will use **TensorFlow** for building and training the model, and **Matplotlib** for visualizing the training results.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## **Loading Image Dataset for Training**

We use **TensorFlow's `image_dataset_from_directory`** to load and preprocess images from a directory.

In [ ]:
data_train = tf.keras.utils.image_dataset_from_directory(
    "train",
    shuffle=True,
    image_size=(180, 180),
    batch_size=32,
    validation_split=False
)

In [ ]:
data_cat = data_train.class_names

## **Loading Validation Dataset**

We use **TensorFlow's `image_dataset_from_directory`** to load and preprocess images for validation.

In [ ]:
data_val = tf.keras.utils.image_dataset_from_directory(
    "validation",
    image_size=(180, 180),
    batch_size=32,
    shuffle=False,
    validation_split=False
)

# **Loading Test Dataset**

We use **TensorFlow's `image_dataset_from_directory`** to load and preprocess images for testing.

In [ ]:
data_test = tf.keras.utils.image_dataset_from_directory(
    "test",
    image_size=(180, 180),
    batch_size=32,
    validation_split=False
)

In [ ]:
plt.figure(figsize=(9,6),dpi=150)
for image, labels in data_train.take(1):
    for i in range(6):
        plt.subplot(2,3,i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.title(data_cat[labels[i]])
        plt.axis("off")

In [ ]:
data_train

## Convolutional Neural Network (CNN) Model

This model is designed for image classification, particularly for datasets like fruits and vegetables. It processes images through multiple convolutional layers and dense layers to extract features and make predictions.

In [ ]:
model = Sequential()

In [ ]:
model.add(layers.Rescaling(1./255))

In [ ]:
model.add(layers.Conv2D(16, 3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D())

In [ ]:
model.add(layers.Conv2D(32, 3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D())

In [ ]:
model.add(layers.Conv2D(64, 3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D())

In [ ]:
model.add(layers.Flatten())

In [ ]:
model.add(layers.Dropout(0.2))

In [ ]:
model.add(layers.Dense(128))

In [ ]:
model.add(layers.Dense(len(data_cat)))

In [ ]:
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

## **Training the CNN Model**

After defining the model architecture, we train it using the **.fit()** function.

In [ ]:
epochs=25
training_results = model.fit(data_train, validation_data=data_val, epochs=epochs, batch_size=32)

## **Visualizing Model Performance**

After training the CNN model, we can analyze its performance using accuracy and loss curves.

In [ ]:
epochs_range = range(epochs)
plt.style.use("dark_background")
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.plot(epochs_range, training_results.history["accuracy"], label="Training Accuracy")
plt.plot(epochs_range, training_results.history["val_accuracy"], label="Validation Accuracy")
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, training_results.history["loss"], label="Training Loss")
plt.plot(epochs_range, training_results.history["val_loss"], label="Validation Loss")
plt.title("Loss")

plt.tight_layout()
plt.legend()
plt.savefig("training_accuracy&loss.png")

## **Image Prediction and Class Probability**

In this section, we load an image, preprocess it, and use the trained model to predict its class. We also calculate the probability of the predicted class and display the result.

In [ ]:
image = "test/spinach/Image_4.jpg"
image = tf.keras.utils.load_img(image, target_size=(180, 180))
image = tf.keras.utils.array_to_img(image)
image = tf.expand_dims(image, 0)

In [ ]:
predict = model.predict(image)

In [ ]:
score = tf.nn.softmax(predict)

In [ ]:
print("Image: {} | Accuracy: {:0.2f}%".format(data_cat[np.argmax(score)], np.max(score)*100))

## **Conclusion**

In this project, we built and trained a Convolutional Neural Network (CNN) model to classify images of fruits and vegetables. The project workflow was as follows:

1. **Dataset Preparation**:
   - We utilized a labeled dataset of fruits and vegetables, which was split into training, validation, and test sets. 
   - The images were preprocessed by resizing them to the target size (180x180), ensuring consistency in input dimensions for the model.

2. **Model Architecture**:
   - The model was built using a Sequential API in TensorFlow with several convolutional layers followed by max-pooling layers to extract features from the images.
   - A Flatten layer was used to convert the 2D feature maps into 1D, and dropout was applied for regularization to prevent overfitting.
   - Finally, the output layer was a Dense layer with the number of units equal to the number of classes (fruits/vegetables), applying a softmax activation function to predict the class probabilities.

3. **Training**:
   - The model was trained using the `fit()` method on the training data for 25 epochs, with a batch size of 32. 
   - We used the validation dataset during training to monitor the model’s performance and ensure generalization.

4. **Evaluation**:
   - After training, the model's performance was evaluated based on accuracy and loss curves, showing how the model learned over time.
   - We used metrics such as accuracy and loss to assess both training and validation performance.

5. **Prediction**:
   - The model was able to successfully predict the class of new images of fruits and vegetables by converting them into the appropriate format and using the trained model to obtain class probabilities.


### **Key Takeaways**:
- This project highlights the power of deep learning, particularly Convolutional Neural Networks (CNNs), in image classification tasks.
- The model was able to effectively classify various types of fruits and vegetables with high accuracy.
- Future improvements could involve experimenting with more advanced techniques like data augmentation or fine-tuning the model for better accuracy.
- This project provides a foundational understanding of how image classification works and can be applied to real-world scenarios in fields like agriculture, food recognition, and even health and safety applications.

Overall, this project demonstrates the process of developing a robust image classification model using TensorFlow and Keras, from data preprocessing to model evaluation and deployment.
